In [1]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 2.3MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=aa3a61a1d44cd0c563a1d3608285cb813fe2e9a12dae1aca7b1e18599d6309f2
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=15dc44c09c0797acac99f3a5c364a4b4ea10e28a9ddacc2ee1b60fa1fbfdaf27
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion_mnist = keras.datasets.fashion_mnist

In [4]:
(train_img,train_class),(test_img,test_class) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
#convert all the values of images(0 to 1)
train_img  = train_img/255.0
test_img = train_img/255.0

In [7]:
train_img[0].shape

(28, 28)

In [9]:
#convert into 2D metrics
train_img = train_img.reshape(len(train_img),28,28,1)
test_img = train_img.reshape(len(test_img),28,28,1)

In [10]:
train_img[0].shape

(28, 28, 1)

In [17]:
#creating model function
def keras_model(hyper_param):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters = hyper_param.Int('conv_1_filter',min_value=32, max_value =128, step=16),
          kernel_size= hyper_param.Choice('conv_1_kernel', values=[3,5]),
          activation = 'relu',
          input_shape = (28,28,1)
      ),
      keras.layers.Conv2D(
          filters = hyper_param.Int('conv_2_filter', min_value =32,max_value=128, step=16),
          kernel_size = hyper_param.Choice('conv_2_kernel',values = [3,5]),
          activation = 'relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units =hyper_param.Int('dense_1_units',min_value = 32, max_value=128,step=16),
          activation = 'relu' 
      ),
      keras.layers.Dense(10, activation ='softmax')
  ])
  model.compile(optimizer = keras.optimizers.Adam(hyper_param.Choice('learning_rate',values=[1e-2, 1e-3])),
                loss ='sparse_categorical_crossentropy',
                metrics =['accuracy'])
  return model

In [13]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [18]:
tuner_search=RandomSearch(keras_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [19]:
tuner_search.search(train_img,train_class,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.3986 - accuracy: 0.8575 - val_loss: 0.2780 - val_accuracy: 0.8938
Epoch 2/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.2564 - accuracy: 0.9058 - val_loss: 0.2474 - val_accuracy: 0.9098
Epoch 3/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.2003 - accuracy: 0.9256 - val_loss: 0.2489 - val_accuracy: 0.9103


Epoch 1/3
1688/1688 [==============================] - 11s 6ms/step - loss: 0.3968 - accuracy: 0.8562 - val_loss: 0.3030 - val_accuracy: 0.8898
Epoch 2/3
1688/1688 [==============================] - 11s 6ms/step - loss: 0.2607 - accuracy: 0.9033 - val_loss: 0.2645 - val_accuracy: 0.9037
Epoch 3/3
1688/1688 [==============================] - 11s 6ms/step - loss: 0.2040 - accuracy: 0.9233 - val_loss: 0.2724 - val_accuracy: 0.9035


Epoch 1/3
1688/1688 [==============================] - 14s 8ms/step - loss: 0.5510 - accuracy: 0.8341 - val_loss: 0.3960 - val_accuracy: 0.8697
Epoch 2/3
1688/1688 [==============================] - 14s 8ms/step - loss: 0.3410 - accuracy: 0.8749 - val_loss: 0.6616 - val_accuracy: 0.7722
Epoch 3/3
1688/1688 [==============================] - 14s 8ms/step - loss: 0.2998 - accuracy: 0.8897 - val_loss: 0.3225 - val_accuracy: 0.8800


Epoch 1/3
1688/1688 [==============================] - 13s 7ms/step - loss: 0.4381 - accuracy: 0.8479 - val_loss: 0.3596 - val_accuracy: 0.8707
Epoch 2/3
1688/1688 [==============================] - 12s 7ms/step - loss: 0.3122 - accuracy: 0.8863 - val_loss: 0.3420 - val_accuracy: 0.8815
Epoch 3/3
1688/1688 [==============================] - 12s 7ms/step - loss: 0.2740 - accuracy: 0.8976 - val_loss: 0.3708 - val_accuracy: 0.8745


Epoch 1/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.3989 - accuracy: 0.8562 - val_loss: 0.3358 - val_accuracy: 0.8740
Epoch 2/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.2595 - accuracy: 0.9034 - val_loss: 0.2574 - val_accuracy: 0.9052
Epoch 3/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.2070 - accuracy: 0.9218 - val_loss: 0.2760 - val_accuracy: 0.9030


INFO:tensorflow:Oracle triggered exit


In [20]:
model = tuner_search.get_best_models(num_models=1)[0]

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 64)        1664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2973792   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 3,013,354
Trainable params: 3,013,354
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(train_img,train_class,epochs=15,validation_split=0.1,initial_epoch=3)

Epoch 4/15
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1551 - accuracy: 0.9424 - val_loss: 0.2558 - val_accuracy: 0.9152
Epoch 5/15
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1183 - accuracy: 0.9561 - val_loss: 0.2960 - val_accuracy: 0.9152
Epoch 6/15
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0882 - accuracy: 0.9674 - val_loss: 0.3121 - val_accuracy: 0.9140
Epoch 7/15
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0648 - accuracy: 0.9762 - val_loss: 0.3540 - val_accuracy: 0.9140
Epoch 8/15
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0525 - accuracy: 0.9808 - val_loss: 0.3692 - val_accuracy: 0.9127
Epoch 9/15
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0400 - accuracy: 0.9857 - val_loss: 0.4245 - val_accuracy: 0.9117
Epoch 10/15
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0337 - accuracy: 0.9879 - val_loss: 0.4623 - val_acc